## Decision Trees

Decision Trees is a popular machine learning algorithm used for both regression and classification tasks. It is a tree-based model that uses a set of rules to make predictions about the target variable.

In a decision tree, the data is split into smaller subsets based on the values of the input features. The tree is built by recursively splitting the data into smaller subsets until a stopping criterion is met, such as reaching a maximum depth or having a minimum number of samples in each leaf node.

Each internal node in the tree represents a test on a feature, and each branch represents the outcome of that test. The leaves of the tree represent the predictions for the target variable.

To build a decision tree, there are several algorithms available, such as ID3, C4.5, CART, etc. The most popular one is the CART (Classification and Regression Tree) algorithm.

For classification tasks, the CART algorithm uses the Gini index to evaluate the quality of a split. The Gini index measures the impurity of a node by calculating the probability of misclassifying a randomly chosen data point in that node. The goal of the algorithm is to minimize the Gini index by selecting the best split among all the possible splits.

For regression tasks, the CART algorithm uses the mean squared error (MSE) to evaluate the quality of a split. The MSE measures the average squared difference between the predicted and actual values of the target variable. The goal of the algorithm is to minimize the MSE by selecting the best split among all the possible splits.

Once the decision tree is built, it can be used to make predictions for new data points by following the path from the root node to a leaf node that corresponds to the values of the input features. The prediction is then the value associated with the leaf node.

Decision trees are popular because they are easy to interpret, and their predictions can be easily explained. However, they can be prone to overfitting if the tree is too deep or if the data has many irrelevant features. To avoid overfitting, techniques such as pruning or ensemble methods like random forests are used.

In [ ]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature = None, threshold = None, left = None, right = None, * , value = None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split = 2, max_depth = 10, n_features = None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value = leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        parent_entropy = self._entropy(y)

        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
        


        

In [ ]:
from random import random
from sklearn import datasets
from sklearn.model_selection import train_test_split

data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

clf = DecisionTree()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

acc =  accuracy(y_test, predictions)
print(acc)